In [1]:
# !pip install transformers datasets boto3 sagemaker --upgrade
# %pip install transformers datasets pipeline faiss-cpu torch


In [2]:
from datasets import load_dataset

dataset = load_dataset("Malikeh1375/medical-question-answering-datasets", "all-processed", split="train")

In [3]:
print(dataset[0])

{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Hey Just wondering.  I am a 39 year old female, pretty smallMy heart rate is around 97 to 106 at rest, and my BP is 140/90 and twice I get 175/118I did visit a doctor because I  didnt feel well past month or twoThen the doctor gave me a heart medicine to take the pulse down and BP  (its still in further examination.)But I wondering what it can be? Do I need the medicine really?  Is that bad ?', 'output': "hello and thank you for using chatbot. i carefully read your question and i understand your concern. i will try to explain you something and give you my opinion. we talk about hypertension if we have mean value that exceeds 140 / 90 mmhg. a person might have high value during emotional and physicals trees so it's mandatory to judge on mean values. usaly hypertension does not give any symptoms but left untreated he slowly modifies the heart. according to heart rhyth

In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="EleutherAI/gpt-neo-125m")

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")

ImportError: cannot import name 'pipeline' from 'transformers' (/Users/rajeshkanna/anaconda3/lib/python3.11/site-packages/transformers/__init__.py)

In [ ]:
def preprocess_function(examples):
    questions = examples["input"]
    contexts = examples["instruction"]
    inputs = [f"question: {q} context: {c}" for q, c in zip(questions, contexts)]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    return model_inputs


tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForCausalLM,GPTNeoForCausalLM

model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m")

In [ ]:
from transformers import RagRetriever, RagTokenizer, RagSequenceForGeneration

# Load the RAG model and tokenizer
rag_tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-nq")
rag_model = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence-nq")

retriever = RagRetriever.from_pretrained("facebook/rag-token-nq", index_name="exact", use_dummy_dataset=True)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
)

trainer = Trainer(
    model=rag_model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()